In realtime data scientist, you may have to check row by row is there data that not meet the several criteria.

In [42]:
import pandas as pd
import dask.dataframe as dd

In [43]:
def filter_valid_spk(df):
    '''
    Checking is a data have a falid status from several conditions.
    '''    
    def checking_other_conditions(spk, invoice_type, invoice_state):
        if spk.endswith('S'):
            # Cheking the length
            if len(spk) == 14:
                # Check is it a null
                if spk:
                    # Check is type == out
                    if invoice_type == 'out':
                        # is state paid or open
                        if invoice_state == 'paid' or invoice_state == 'open':
                            spk_status = True
                        else:
                            spk_status = False
                    else:
                        spk_status = False
                else:
                    spk_status = False
            else:
                spk_status = False
        else:
            spk_status = False
        return spk_status


    def filter_valid_spk(row):
        spk = row['s']
        if spk:
            invoice_type = row['type']
            invoice_state = row['state']

            spk = spk.split(',')
            new_spk = []
            for s in spk:
                new_spk.append(s.strip())
            spk = new_spk
            spk_status = False
            for s in spk:
                spk_status_s = checking_other_conditions(s, invoice_type, invoice_state)
                spk_status = spk_status or spk_status_s
        else:
            spk_status = False
        return spk_status
    
    df['status'] = df.apply(filter_valid_spk, axis=1, meta=bool)
    
    # Fileter to status == True
    df = df[df['status']==True]

    return df

In [44]:
df = pd.DataFrame()
df['s'] = [
    '1404019000014S, 404019000033S, 404019000017S',
    None,
    'invalid_spk',
    '1404019000036S, 1404019000035S',
    '1404019000002S, 404019000022S, 404019000016S',
    'long-long-long-long-invalid-S',
    '404019000012S',
    '404019000032S',
    '1404019000040S',
    '1404019000038S',
    '1404019000024S',
    '1404019000015S',
    '1404019000039S',
    '1404019000042S',
    '1404019000006S',
    '1404019000027S',
    '1404019000026S']
df['type'] = ['out',
               'out', 
               'out', 
               'out',
               'out', 
               'out', 
               'out', 
               'out',
               'out',
               'out',
               'out',
               'out', 
               'out', 
               'out',
               'out', 
               'in', 
               'in']
df['state'] = ['paid',
                'open', 
                'paid', 
                'open',
                'paid', 
                'open', 
                'paid', 
                'open',
                'paid',
                'open', 
                'paid', 
                'open',
                'paid',
                'not_paid', 
                'not_open', 
                'open',
                'paid']

df = dd.from_pandas(df, npartitions=2)
df.head(100)


/Users/thomas/miniconda3/lib/python3.6/site-packages/dask/dataframe/core.py:5739: UserWarning: Insufficient elements for `head`. 100 elements requested, only 9 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,s,type,state
0,"1404019000014S, 404019000033S, 404019000017S",out,paid
1,None,out,open
2,invalid_spk,out,paid
3,"1404019000036S, 1404019000035S",out,open
4,"1404019000002S, 404019000022S, 404019000016S",out,paid
5,long-long-long-long-invalid-S,out,open
6,404019000012S,out,paid
7,404019000032S,out,open
8,1404019000040S,out,paid


In [45]:
df = filter_valid_spk(df)
df.head(100)


/Users/thomas/miniconda3/lib/python3.6/site-packages/dask/dataframe/core.py:5739: UserWarning: Insufficient elements for `head`. 100 elements requested, only 4 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,s,type,state,status
0,"1404019000014S, 404019000033S, 404019000017S",out,paid,True
3,"1404019000036S, 1404019000035S",out,open,True
4,"1404019000002S, 404019000022S, 404019000016S",out,paid,True
8,1404019000040S,out,paid,True
